In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor


In [41]:
train_data = TabularDataset("./data/Planning_Combined.csv")
# subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
# train_data = train_data.sample(n=subsample_size, random_state=0)
train_data = train_data.drop(columns=["Fulfilled", "Product", "Delivery Week", "Quantity", "Cost", "Land Ownership"])  # delete label column to prove we're not cheating
train_data.head()



Loaded data from: ./data/Planning_Combined.csv | Columns = 10 / 10 | Rows = 1592 -> 1592


,Producer Code,Product ID,Water Access,FulfilPerc
0,PBY,3166,False,0.0
1,SGC,3188,NaN,1.0
2,LXF,3613,NaN,1.0
3,HAF,3755,False,0.0
4,LXF,3132,NaN,1.0


In [42]:
label = 'FulfilPerc'
print("Summary of class variable: \n", train_data[label].describe())


Summary of class variable: 
 count    1592.000000
mean        0.513214
std         0.483369
min         0.000000
25%         0.000000
50%         0.632456
75%         1.000000
max         1.000000
Name: FulfilPerc, dtype: float64


In [43]:
save_path = 'agModels-aly'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-aly/"
AutoGluon Version:  0.6.2
Python Version:     3.9.16
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.2.0: Fri Nov 11 02:03:51 PST 2022; root:xnu-8792.61.2~4/RELEASE_ARM64_T6000
Train Data Rows:    1592
Train Data Columns: 3
Label Column: FulfilPerc
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (1.0, 0.0, 0.51321, 0.48337)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9030.48 MB
	Train Data (Original)  Memory Usage: 0.16

[1000]	valid_set's rmse: 0.44588
[2000]	valid_set's rmse: 0.443742


	-0.4434	 = Validation score   (-root_mean_squared_error)
	4.37s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	-0.44	 = Validation score   (-root_mean_squared_error)
	2.23s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.455	 = Validation score   (-root_mean_squared_error)
	0.21s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: CatBoost ...
	-0.4546	 = Validation score   (-root_mean_squared_error)
	3.56s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.4554	 = Validation score   (-root_mean_squared_error)
	0.19s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
	-0.4654	 = Validation score   (-root_mean_squared_error)
	0.91s	 = T

In [44]:
test_data = TabularDataset('./data/Planning_Combined.csv')
test_data = test_data.drop(columns=["Fulfilled"])  # delete fulfilled col
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()


Loaded data from: ./data/Planning_Combined.csv | Columns = 10 / 10 | Rows = 1592 -> 1592


,Delivery Week,Producer Code,Product ID,Product,Quantity,Cost,Water Access,Land Ownership
0,10/11/2021,PBY,3166,"Potatoes, Red",10,5000,False,False
1,10/11/2021,SGC,3188,"Squash, Winter, Butternut",25,2465,NaN,NaN
2,10/11/2021,LXF,3613,"Beets, Bulk",18,2631,NaN,NaN
3,10/11/2021,HAF,3755,"Carrots, Orange Bulk",12,3162,False,True
4,10/11/2021,LXF,3132,"Carrots, Rainbow Bulk",60,3162,NaN,NaN


In [45]:
predictor.leaderboard(test_data)

                  model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       RandomForestMSE   -0.376519  -0.455035        0.040261       0.024349   0.212043                 0.040261                0.024349           0.212043            1       True          5
1         ExtraTreesMSE   -0.376677  -0.455378        0.038962       0.022654   0.189362                 0.038962                0.022654           0.189362            1       True          7
2               XGBoost   -0.386723  -0.439282        0.053809       0.003369   7.697358                 0.053809                0.003369           7.697358            1       True          9
3              LightGBM   -0.391919  -0.439967        0.006918       0.002419   2.226134                 0.006918                0.002419           2.226134            1       True          4
4   WeightedEnsemble_L2   -0.393301  -0.

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,-0.376519,-0.455035,0.040261,0.024349,0.212043,0.040261,0.024349,0.212043,1,True,5
1,ExtraTreesMSE,-0.376677,-0.455378,0.038962,0.022654,0.189362,0.038962,0.022654,0.189362,1,True,7
2,XGBoost,-0.386723,-0.439282,0.053809,0.003369,7.697358,0.053809,0.003369,7.697358,1,True,9
3,LightGBM,-0.391919,-0.439967,0.006918,0.002419,2.226134,0.006918,0.002419,2.226134,1,True,4
4,WeightedEnsemble_L2,-0.393301,-0.434256,0.121438,0.040241,18.570700,0.002001,0.000160,0.117243,2,True,12
5,LightGBMLarge,-0.396512,-0.442456,0.005086,0.002017,7.755696,0.005086,0.002017,7.755696,1,True,11
6,LightGBMXT,-0.405980,-0.443380,0.031337,0.006563,4.373518,0.031337,0.006563,4.373518,1,True,3
7,CatBoost,-0.415212,-0.454585,0.012719,0.002363,3.559114,0.012719,0.002363,3.559114,1,True,6
8,NeuralNetFastAI,-0.448638,-0.465415,0.020138,0.006408,0.913262,0.020138,0.006408,0.913262,1,True,8
9,NeuralNetTorch,-0.478633,-0.479835,0.009195,0.004666,0.558707,0.009195,0.004666,0.558707,1,True,10


In [46]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Evaluation: root_mean_squared_error on test data: -0.3933012199900303
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.3933012199900303,
    "mean_squared_error": -0.15468584964564622,
    "mean_absolute_error": -0.3508901309108547,
    "r2": 0.3375307616087917,
    "pearsonr": 0.5998877878579456,
    "median_absolute_error": -0.3297884464263916
}


Predictions:  
 0       0.270469
1       0.592922
2       0.568352
3       0.468154
4       0.196764
          ...   
1587    0.416508
1588    0.675176
1589    0.075513
1590    0.706524
1591    0.093120
Name: FulfilPerc, Length: 1592, dtype: float32


In [51]:
predict_data = TabularDataset('./data/test.csv')
predict_data = predict_data.drop(columns=["Fulfilled", "FulfilPerc"])  # delete fulfilled col
predict_data.head()

predictor.predict(predict_data)


Loaded data from: ./data/test.csv | Columns = 10 / 10 | Rows = 1 -> 1
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


0    0.67425
Name: FulfilPerc, dtype: float32